In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 9.6MB/s 
     |████████████████████████████████| 460kB 40.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import os

In [9]:
from google.colab import drive

In [17]:
from konlpy.tag import Okt
from gensim.models import Word2Vec, KeyedVectors
from gensim.models import FastText
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams, pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM
from keras.models import load_model

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [10]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## 1. 데이터 로드

In [11]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/project_data/Crawling_Lyrics.csv')

In [12]:
df.head()

,title,singer,lyrics
0,너희는 (Feat. 김광률),이은영,너희는 하나님의 택하신\n거룩하고 사랑스러운 자니 \n긍율과 자비와 \n겸손과 온유...
1,봄날은 간다 (Bonus Track),김윤아,눈을 감으면 문득 그리운 날의 기억 \n아직까지도 마음이 저려 오는 건 \n\n그건...
2,Going Home,김윤아,집으로 돌아가는 길에\n지는 햇살에 마음을 맡기고\n나는 너의 일을 떠올리며\n수많...
3,야상곡 (夜想曲),김윤아,바람이 부는 것은 더운 내 맘 삭여주려 \n계절이 다 가도록 나는 애만 태우네 \n...
4,길,김윤아,아무도 가르쳐 주지 않아\n이 길이 옳은지 다른 길로 가야 할지\n난 저길 저 끝에...


## 2. 데이터 전처리

In [13]:
df.dropna(axis=0, inplace = True)

In [14]:
lyrics = list(df['lyrics'])

In [20]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lyrics)
vocab_size = len(tokenizer.word_index) + 1

print('단어 집합의 크기: %d' %vocab_size)

단어 집합의 크기: 15618


In [ ]:
tokenizer.word_index

- 훈련데이터 생성

In [22]:
%%time

sequences = list()

for lyric in lyrics:
  for line in lyric.split('\n'):
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)) :
      sequence = encoded[:i+1]
      sequences.append(sequence)

CPU times: user 258 ms, sys: 1.72 ms, total: 259 ms
Wall time: 265 ms


In [23]:
print('학습에 사용할 샘플의 개수: %d' %len(sequences))

학습에 사용할 샘플의 개수: 65895


In [24]:
print(sequences)

[[3048, 3049], [3048, 3049, 3050], [3051, 1611], [3051, 1611, 3052], [5412, 5413], [5414, 5415], [5414, 5415, 5416], [5414, 5415, 5416, 5417], [3048, 3049], [3048, 3049, 3050], [3051, 1611], [3051, 1611, 3052], [206, 5418], [49, 32], [49, 32, 312], [49, 32, 312, 91], [49, 32, 312, 91, 5419], [1159, 962], [1159, 962, 3053], [1159, 962, 3053, 2519], [1159, 962, 3053, 2519, 3993], [1159, 962, 3053, 2519, 3993, 3994], [3054, 3995], [3054, 3995, 3996], [3054, 3995, 3996, 1436], [3054, 3995, 3996, 1436, 2099], [3048, 3049], [3048, 3049, 3050], [3051, 1611], [3051, 1611, 3052], [5412, 5413], [5414, 5415], [5414, 5415, 5416], [5414, 5415, 5416, 5417], [3048, 3049], [3048, 3049, 3050], [3051, 1611], [3051, 1611, 3052], [206, 5418], [49, 32], [49, 32, 312], [49, 32, 312, 91], [49, 32, 312, 91, 5419], [1159, 962], [1159, 962, 3053], [1159, 962, 3053, 2519], [1159, 962, 3053, 2519, 3993], [1159, 962, 3053, 2519, 3993, 3994], [3054, 3995], [3054, 3995, 3996], [3054, 3995, 3996, 1436], [3054, 3995, 

In [25]:
max_len = max(len(l) for l in sequences)

print('샘플의 최대 길이: {}' .format(max_len))

샘플의 최대 길이: 74


In [26]:
sequences = pad_sequences(sequences, maxlen = max_len, padding = 'pre')

print(sequences)

[[   0    0    0 ...    0 3048 3049]
 [   0    0    0 ... 3048 3049 3050]
 [   0    0    0 ...    0 3051 1611]
 ...
 [   0    0    0 ...    0  729  201]
 [   0    0    0 ...  729  201   16]
 [   0    0    0 ...  201   16   12]]


In [27]:
np.shape(sequences)

(65895, 74)

- 각 샘플의 마지막 단어를 레이블로 분리

In [28]:
sequences = np.array(sequences)

X = sequences[:, :-1]
y = sequences[:, -1]

In [29]:
np.shape(X) , np.shape(y)

((65895, 73), (65895,))

- 레이블에 대해 원-핫 인코딩

In [30]:
y = to_categorical(y, num_classes=vocab_size)

## 3. Modeling

In [31]:
output_dim = 100


model = Sequential()

model.add(Embedding(vocab_size, output_dim = output_dim,
                    input_length = max_len -1))

model.add(LSTM(256))
model.add(Dense(vocab_size, activation = 'softmax'))

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 73, 100)           1561800   
_________________________________________________________________
lstm (LSTM)                  (None, 256)               365568    
_________________________________________________________________
dense (Dense)                (None, 15618)             4013826   
Total params: 5,941,194
Trainable params: 5,941,194
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [ ]:
# 약 35분 소요
%%time
history = model.fit(X, y, 
                    epochs = 50, verbose = 2)

In [34]:
model_loaded = load_model('/content/drive/MyDrive/Colab Notebooks/Project/001_RNN_Generation.h5')

## 4. 가사 생성 함수 정의

In [38]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
  init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
  sentence = ''
  for _ in range(n):
    # 현재 단어에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([current_word])[0] 
    # 데이터에 대한 패딩
    encoded = pad_sequences([encoded], maxlen=max_len, padding='pre') 
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
    result = model.predict_classes(encoded, verbose=0)

    for word, index in tokenizer.word_index.items(): 
      if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
        break # 해당 단어가 예측 단어이므로 break
    current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
    sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
  # for문이므로 이 행동을 다시 반복

  sentence = init_word + sentence
  return sentence

In [39]:
print(sentence_generation(model_loaded, tokenizer, '사랑', 12))

사랑 언제나 이번이 마지막이라며 처음인 듯 찾아오니까 숨긴 미소면 나도 나도 나의 진실을
